## 11.2 Retrofit - praca z URL

W poprzedniej aplikacji uzyskaliśmy dostęp do wszystkich postów, zazwyczaj potrzebujemy tylko jakiś podzbiór wszystkich dostępnych danych. W tym celu posługujemy się odpowiednimi parametrami w adresie URL. Ponownie wykorzystamy [**JSONPlaceholder**](https://jsonplaceholder.typicode.com/) - w sekcji **Rooutes** mamy podane różne rodzaje metod `HTTP`
- GET	/posts
- GET	/posts/1
- GET	/posts/1/comments
- GET	/comments?postId=1
- POST	/posts
- PUT	/posts/1
- PATCH	/posts/1
- DELETE	/posts/1

W tym przykładzie wyświetlimy wszystkie komentarze pod postem o zadanym identyfikatorze. Możemy to wykonać wykorzystując */posts/1/comments*
- posts - zwraca wszystkie posty
- 1 - identyfikator posta
- comments - zwraca wszystkie komentarze

Pamiętamy aby aplikacja miała dostęp do internetu oraz odpowiednie zależności.

Rozpoczniemy od modelu - format komentarzy wygląda następująco

In [ ]:
{
"postId": 1,
"id": 1,
"name": "id labore ex et quam laborum",
"email": "Eliseo@gardner.biz",
"body": "laudantium enim ..."
},

Więc tworzymy klasę o odpowiednich polach

In [ ]:
data class Comment(
    val postId: Int,
    val id: Int,
    val name: String,
    val email: String,
    @SerializedName("body")
    val text: String
)


Utwórzmy nasz interfejs

In [ ]:
interface PlaceholderService {
    @GET("posts/1/comments")
    fun getComments(): Call<List<Comment>>
}

Utwórzmy `Retrofit` - tym razem jako singleton

In [ ]:
object RetrofitFactory {

    private const val url = "https://jsonplaceholder.typicode.com/"

    val service: PlaceholderService by lazy {
        Retrofit.Builder()
            .baseUrl(url)
            .addConverterFactory(GsonConverterFactory.create())
            .build().create(PlaceholderService::class.java)
    }
}

W aktywności głównej kod w niewielkim stopniu różnie się od poprzedniego przykładu

In [ ]:
val textView = findViewById<TextView>(R.id.textView)

val service: PlaceholderService = RetrofitFactory.service

val call = service.getComments("comments?postId=3")

call.enqueue(object : Callback<List<Comment>?> {
    @SuppressLint("SetTextI18n")
    override fun onResponse(
        call: Call<List<Comment>?>,
        response: Response<List<Comment>?>
    ) {
        if (response.isSuccessful) {
            val comments = response.body()
            comments?.forEach(Consumer { comment: Comment ->
                val content = StringBuilder()
                content.append("id: ").append(comment.id).append("\n")
                    .append("PostId: ").append(comment.postId).append("\n")
                    .append("name: ").append(comment.name).append("\n")
                    .append("email: ").append(comment.email).append("\n")
                    .append("text: ").append(comment.text).append("\n\n")
                textView.append(content)
            })
        } else textView.text = "Code: " + response.code()
    }

    override fun onFailure(call: Call<List<Comment>?>, t: Throwable) {
        textView.text = t.message
    }
})

W odpowiedzi dostaniemy listę wszysktich komentarzy pod postem o identyfikatorze równym 1.

### **@Path**

Jeżeli chcemy napisać metodę przyjmującą jako parametr `id` posta, musimy wykorzystać odpowiednią adnotację - zmodyfikujmy metodę `getComments` w interfejsie `PlaceholderService`

In [ ]:
@GET("posts/{id}/comments")
fun getComments(@Path("id") postId: Int): Call<List<Comment>>

Dzięki zastosowaniu adnotacji `@Path` przekazujemy informację o zastosowaniu parametru metody jako składowej adresu `URL`. Tutaj musimy zwrócić uwagę na parametr `@Path` `"id"` - musi on być zgodny z częścią parametru adnotacji `@GET`, który podajemy w nawiasach `{id}`. Teraz wywołanie metody wygląda następująco

In [ ]:
val call = service.getComments(3)

### **@Query**

Mamy również drugi sposób przekazania argumentu i uzyskania tej samej informacji - */comments?postId=1*. Dostaniemy dokładnie ta samą informację - wszystkie komentarza pod postem o zadanym `id`. Tym razem musimy wykorzystać adnotację `@Query` - zapytanie jest rozpoczynane symbolem `?`.

Dodajmy metodę `getCommentsFromQuery`

In [ ]:
@GET("comments")
fun getCommentsFromQuery(@Query("postId") postId: Int): Call<List<Comment>>

Tutaj parametr adnotacji `@Query` musi odpowiadać nazwie parametru (lub metody) obecnej w zapytaniu - tutaj będzie to */comments?postId=1*. Pozostałe elementy zapytania (znak rozpoczynający zapytanie oraz symbol `=`) zostanie dodany automatycznie. Metodę wywołujemy tak samo jak poprzednią

In [ ]:
val call = service.getCommentsFromQuery(3);

Możemy również podać wiele parametrów - ich nazwy i wszystkie możliwości znadziemy w dokumentacji na stronie z  której korzystamy.

Chcemy uzyskać listę wszystkich komentarzy pod postem o zadanych `id`, posortowane po identyfikatorze malejąco. `url` będzie wyglądał następująco
- */comments?postId=1&_sort=id&_order=desc* - znak `&` rozdziela parametry

Napiszmy odpowiednią metodę

In [ ]:
@GET("comments")
fun getSortedComments(
    @Query("postId") postId: Int,
    @Query("_sort") sort: String,
    @Query("_order") order: String
): Call<List<Comment>>

Wywołanie metody

In [ ]:
val call = service.getSortedComments(2, "id", "desc")

Zwróćmy uwagę że parametr sortowania podajemy jakoi `String`.

Jeżeli chcemy dostać wszystkie komentarze posortowane malejąco po `id`, możemy przekazać jako parametr `posrId` wartość `null`

Teraz chcąc wszystkie komentarze, możemy wywołać to funkcję

In [ ]:
val call = service.getSortedComments(null, "id", "desc")

`null` można podać jako każdy argument - chcąc dostać wszystkie komentarze możemy wywołać metodę

In [ ]:
val call = service.getSortedComments(null, null, null)

Jeżeli chcemy dostać listę komentarzy z kilku postów, możemy zadeklarować metodę przyjmującą tablicę (lub listę) identyfikatorów

In [ ]:
@GET("comments")
fun getSortedCommentsFromPosts(
    @Query("postId") postsId: List<Int>,
    @Query("_sort") sort: String,
    @Query("_order") order: String
): Call<List<Comment>>

Wtedy chcąc otrzymać posortowane komentarze z kilku postów, możemy metodę wywołać następująco

In [ ]:
Call<List<Comment>> call = service.getSortedCommentsFromPosts(
    listOf(1, 3, 6, 7), "id", "desc");

### **@QueryMap**

Jeżeli chcemy napisać metodę przyjmującą dowolną ilość parametrów, których nie określamy w definicji samej funkcji, możemy wykorzystać adnotację `@QueryMap`

In [ ]:
@GET("comments")
fun getComments(
    @QueryMap param: Map<String, String>
): Call<List<Comment>>

Nasza metoda przyjmuje `Map` - pary klucz-wartość, gdzie kluczem typu `String` jest nazwa parametru. Wartości też podajemy jako `String`. Teraz w głównej aktywności możemy utworzyć mapę pożądanych parametrów i wykorzystać ją jako parametr funkcji.

In [ ]:
val param: MutableMap<String, String> = HashMap()
param["postId"] = "1"
param["_sort"] = "id"
param["_order"] = "desc"

val call = service.getComments(param);

### **@Url**

Jeżeli adres jest skomplikowany z większą ilością parametrów, możemy chieć przekazać sam `url` jako parametr funkcji.

In [ ]:
@GET
fun getComments(
    @Url url: String
): Call<List<Comment>>

Wtedy w wywołaniu metody `getComments` podajemy cały adres `url`

In [ ]:
val call = service.getComments("comments?postId=3");

Możemy również podać

In [ ]:
val call = service.getComments(
    "https://jsonplaceholder.typicode.com/comments?postId=3");

Na tym zakończymy omawianie podstaw pracy z adresami `url` oraz `GET`